# NTRU Encrypt

O sistema criptográfico NTRU é um sistema de chave pública baseado em reticulados, tendo sido o primeiro deste tipo. 

## Parâmetros

Num sistema criptográfico NTRU, **f** (e **g**, se necessário) são as chaves privadas, enquanto h
é a chave pública. Essas chaves podem ser geradas através do seguinte algoritmo:

## Algoritmo de Geração de chaves

INPUT: Um conjunto de parâmetros $Param = \{N, p, q, d\}$ e uma $seed$.

1. Instanciar $Sampler$ com $\tau(d + 1, d)$ e com a $seed$;
2. $f ← Sampler$
3. se $f$ não é invertível $mod q$ então retornar ao passo 2
4. $g ← Sampler$
5. $h = g/(pf + 1) mod q$

OUTPUT: Chave pública $h$ e chave secreta $(pf, g)$

## Algoritmo de Cifragem

INPUT: Chave pública h, mensagem msg de comprimento mlen, um conjunto de parâmetros $Param$ e a $seed$.
1. $m = Pad(msg, seed)$
2. $rseed = Hash(m|h)$
3. Instanciar $Sampler$ com $T$ e com $rseed$;
4. $r ← Sampler$
5. $t = r ∗ h$
6. $tseed = Hash(t)$
7. Instanciar $Sampler$ com $T$ e com $tseed$;
8. $m_{mask} ← Sampler$
9. $m' = m − m_{mask}(mod p)$
10. $c = t + m$

OUTPUT: Criptograma **c**

O algoritmo acima usa um **método de *padding*** para lidar com
entropia insuficiente potencial de uma mensagem. Supondo que o tamanho da mensagem é válido e menor que $(N - 173)$ *bits*, o algoritmo *padding* funciona da seguinte maneira:

1. Converter $msg$ numa *string* de *bits*. Cada *bit* forma um coeficiente binário para a parte inferior do polinómio $m$, partindo do coeficiente 0.

2. Os últimos $167$ coeficientes de $m (x)$ são escolhidos aleatoriamente de $\{−1, 0, 1\}$ (com um *input* *seed* ). O que dá mais de $256$ bits de entropia.

3. O comprimento da msg é convertido numa *string* binária de 8 *bits* e forma os últimos $173$ a $168$ coeficientes de $m (x)$.

## Algorithm de Decifragem

INPUT: Chave secreta $f$, chave pública $h$, criptograma $c$, e o conjunto de parâmetros $Param$.
1. $m' = f ∗ c (mod p)$
2. $t = c − m$
3. $tseed = Hash(t)$
4. Instanciar $Sampler$ com $T$ e com $tseed$;
5. $m_{mask} = Sampler$
6. $m = m' + m_{mask} (mod p)$
7. $rseed = Hash(m|h)$
8. Instanciar $Sampler$ com $T$ e com $rseed$;
9. $r ← Sampler$
10. $msg, mlen = Extract(m)$
11. se $p · r ∗ h = t$ então
12. $result = msg, mlen$
13. caso contrário
14. $result = ⊥ $

OUTPUT: $result$

No algoritmo acima a operação $Extrair ()$ corresponde ao inverso de $Pad ()$. Emite uma mensagem $m$ e seu comprimento $mlen$.

In [21]:
class Sampler:
    def __init__(self, N, q, d, seed):
        #TODO: use seed
        Z.<x>  = ZZ[]
        Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1)
        self.Q = Q
    
    def gen_value(self, n):
        return self.Q(gen_bstring(n))


def gen_bstring(n):
    '''Generate a random bit string
    '''
    return  [choice([-1,0,1]) for k in range(n)]

def rnd_modq(f, q):
    '''Round f mod q
    '''
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

def rnd_modp(l, p):
    '''Round l mod p
    '''
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

def pad_msg(msg,seed):
    ''' Padding method
    '''
    mlen = len(msg)
    if mlen < (N-173):
        pass
            
def extract_msg(msg):
    ''' Inverse of the padding function
    '''
    

In [22]:
from hashlib import sha256

class NTRU(object):
    def __init__(self, d):
        self.d = d
        self.N = next_prime(1 << self.d)
        self.p = 3
        self.q = next_prime(self.p*self.N)
        self.keygen()
        
    def keygen(self):
        smpl = Sampler(self.N, self.q, self.d, gen_bstring(self.N))
        F = smpl.gen_value(self.N)
        f = 1 + self.p*F
        while not f.is_unit():
            F = Q(gen_bstring())
            f = 1 + self.p*F
        
        G = smpl.gen_value(self.N)
        g = self.p*G
        h = rnd_modq(f^(-1) * g, self.q)
        self.chvpriv = (f,g)
        self.chvpub = h
        
    def encrypt(self, msg, seed):
        padded_msg = pad_msg(msg, seed)
        rseed = int(sha256(padded_msg + self.chvpub).hexdigest(), 16)
        smpl = Sampler(self.N, self.q, self.d, rseed)
        r = smpl.gen_value(self.N)
        t = r*self.chvpub
        tseed = int(sha256(t).hexdigest(), 16)
        smpl = Sampler(self.N, self.q, self.d, tseed)
        msg_mask = smpl.gen_value(self.N)
        masked_msg = padded_msg - msg_mask
        crypt = t + masked_msg
        return crypt

    def decrypt(self, crypt):
        masked_msg = mod(self.chvpriv[0] * crypt, self.p)
        t = crypt - masked_msg
        tseed = int(sha256(self.chvpub).hexdigest(), 16)
        smpl = Sampler(self.N, self.q, self.d, tseed)
        msg_mask = smpl.gen_value(self.N)
        padded_msg = masked_msg + msg_mask
        rseed = int(sha256(padded_msg + self.chvpub).hexdigest(), 16)
        smpl = Sampler(self.N, self.q, self.d, rseed)
        r = smpl.gen_value(self.N)
        msg, mlen = extract_msg(padded_msg)
        if self.p*r*self.chpub == t:
            result = msg,mlen
        else:
            result = "Error"
        return result

In [23]:
K = NTRU(6)
m = gen_bstring(6)
e = K.encrypt(m)
m == K.decrypt(e)

TypeError: unsupported operand parent(s) for -: '<type 'function'>' and 'Integer Ring'